# Stock Trading Scheme

#### Importing packages

In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy
import quandl
from stockAnalysisTools import numToString

#### The cell below will be used to pull all the stock data we want and make a dictionary for accessing it

In [27]:
quandl.ApiConfig.api_key = "d1JcayaRuvKSaq7GaEHs"
aapl = "aapl"
goog  = "goog"
stockList = [aapl, goog]
finData = dict()

for ticker in stockList:
    finData[ticker] = quandl.get("WIKI/"+ticker)["Adj. Close"]

#### Creating an object to hold portfolio data

In [37]:
class Portfolio(object):
    names = list()
    stocks = list()
    uninvestedMoney = 0
    
    def addStock(self, name):
        
        if(name not in finData.keys()):
            print("We haven't pulled up data for this stock, so it can't be added")
            return

        self.names.append(name)
        self.stocks.append(0)
    
    def addMoney(self, money):
        
        self.uninvestedMoney += money
        
    def reset(self):
        self.uninvestedMoney = 0
        for i in range(len(self.stocks)):
            self.stocks[i] = 0
            
    def buyStock(self, name, date = 0, price = 0, num = 0, noLimit=True):
        if name not in self.names:
            print("Stock you want to buy isn't in the self")
            print("It must be added to the portfolio before you can buy")
            return

        if(date == 0):
            price_of_stock = price
        else:
            price_of_stock = finData[name].loc[date+" 00:00:00"]

        if(noLimit):
            numOfStock = self.uninvestedMoney/price_of_stock
            self.uninvestedMoney -= numOfStock * price_of_stock
            self.stocks[self.names.index(name)] += numOfStock
        else:
            if(num * price_of_stock() > self.uninvestedMoney):
                print("Can't buy this much stock")
            else:
                self.uninvestedMoney -= num * price_of_stock
                self.stocks[self.names.index(name)] += num 

    def sellStock(self, name, date = 0, price = 0, num = 0, noLimit=True):
        if name not in self.names:
            print("Stock you want to buy isn't in the self")
            print("It must be added to the portfolio before you can buy")
            return

        if(date == 0):
            price_of_stock = price
        else:
            price_of_stock = finData[name].loc[date+" 00:00:00"]

        numOfStock = self.stocks[self.names.index(name)]

        if(noLimit):
            self.stocks[self.names.index(name)] = 0

            self.uninvestedMoney += numOfStock * price_of_stock
        else:
            if(numOfStock - num < 0):
                print("Can't sell this much stock")
            else:
                self.uninvestedMoney += num * price_of_stock
                self.stocks[self.names.index(name)] -= num
        

In [7]:
# aaplAdjClose = aaplData["Adj. Close"]
print(finData["aapl"].loc['2001-07-01':'2018-07-01'])

Date
2001-07-02      1.535742
2001-07-03      1.531886
2001-07-05      1.490119
2001-07-06      1.415581
2001-07-09      1.458633
2001-07-10      1.358392
2001-07-11      1.448352
2001-07-12      1.565300
2001-07-13      1.596786
2001-07-16      1.539597
2001-07-17      1.612850
2001-07-18      1.335902
2001-07-19      1.282569
2001-07-20      1.283854
2001-07-23      1.255581
2001-07-24      1.226666
2001-07-25      1.186826
2001-07-26      1.194537
2001-07-27      1.218312
2001-07-30      1.216384
2001-07-31      1.207388
2001-08-01      1.224738
2001-08-02      1.273573
2001-08-03      1.253011
2001-08-06      1.229236
2001-08-07      1.236947
2001-08-08      1.214457
2001-08-09      1.224095
2001-08-10      1.222168
2001-08-13      1.226666
                 ...    
2018-02-13    164.340000
2018-02-14    167.370000
2018-02-15    172.990000
2018-02-16    172.430000
2018-02-20    171.850000
2018-02-21    171.070000
2018-02-22    172.600000
2018-02-23    175.555000
2018-02-26    178.97

#### Creating Portfolio

In [39]:
my_portfolio = Portfolio()
my_portfolio.addStock(aapl)

In [30]:
#This cell is used for testing
aaplAdjClose = aaplData["Adj. Close"]["2018-03-27 00:00:00"]
print(aaplAdjClose)
# print(aaplAdjClose.keys()[0])

168.34


#### Simple Trading Strategy

In [40]:
#portfolio reset
my_portfolio.reset()
my_portfolio.addMoney(100)
odd = True
startDate = '2001-07-01'
endDate = '2018-03-27'
for adj_price in finData[aapl].loc[startDate: endDate]:
    if(odd):
        my_portfolio.buyStock(aapl, price = adj_price)
    else:
        my_portfolio.sellStock(aapl, price = adj_price)
        
    odd = (not odd)
    
sellStock(aapl, my_portfolio, date = endDate)

In [41]:
print("Value of Investment $"+ numToString(my_portfolio.uninvestedMoney))

Value of Investment $355.03061640616050454


#### Rate of Return Series Generated

In [35]:
returnSer = deepcopy(finData)
for key in returnSer.keys():
    for i in range(len(returnSer[key])-1, 0, -1):
        returnSer[key][i] = ((returnSer[key][i]/returnSer[key][i-1]) - 1)*100
    
    returnSer[key][0] = np.nan
    print(key)
    print(returnSer[key])
        
# print(returnSer[aapl])    

aapl
Date
1980-12-12         NaN
1980-12-15   -5.217391
1980-12-16   -7.339450
1980-12-17    2.455446
1980-12-18    2.937766
1980-12-19    6.083365
1980-12-22    4.884956
1980-12-23    4.218697
1980-12-24    5.246114
1980-12-26    9.230769
1980-12-29    1.408451
1980-12-30   -2.444444
1980-12-31   -2.818907
1981-01-02    1.084090
1981-01-05   -2.173913
1981-01-06   -4.444444
1981-01-07   -4.248062
1981-01-08   -2.040155
1981-01-09    5.388430
1981-01-12   -0.815558
1981-01-13   -3.542062
1981-01-14    0.426230
1981-01-15    2.024159
1981-01-16   -0.800000
1981-01-19    6.032258
1981-01-20   -3.011865
1981-01-21    1.944793
1981-01-22    1.138462
1981-01-23   -0.365075
1981-01-26   -1.526718
                ...   
2018-02-13    1.001782
2018-02-14    1.843739
2018-02-15    3.357830
2018-02-16   -0.323718
2018-02-20   -0.336368
2018-02-21   -0.453884
2018-02-22    0.894371
2018-02-23    1.712051
2018-02-26    1.945259
2018-02-27   -0.324077
2018-02-28   -0.151354
2018-03-01   -1.751628
2